In [1]:
import mne.time_frequency
import mne
from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, channel_outlier_marker, crop_empty_data, outliers_to_nan
from ieeg.calc.scaling import rescale
import os
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np


In [ ]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")

print(subjects)
# for sub in subjects:

sub='D0057' #why does this not work in the for loop???
# Load the data
filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                           extension='.edf', desc='clean', preload=False) #get line-noise filtered data
print(filt)

## Crop raw data to minimize processing time
good = crop_empty_data(filt)

# good.drop_channels(good.info['bads'])
good.info['bads'] = channel_outlier_marker(good, 3, 2)
good.drop_channels(good.info['bads'])
# good.info['bads'] += channel_outlier_marker(good, 4, 2)
# good.drop_channels(good.info['bads'])
good.load_data()

ch_type = filt.get_channel_types(only_data_chs=True)[0]
good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

# Remove intermediates from mem
# good.plot()

## epoching and trial outlier removal

save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

  



### testing to get avg baseline

In [16]:
EpochsTFRshape = 0,176,46,50

print(np.empty(shape=EpochsTFRshape).shape)

(0, 176, 46, 50)


In [41]:
trialTypes = ["i75s25", "c75s25", "i75r25", "c75r25", "i75s75", "c75s75", "i75r75", "c75r75",
              "i25s25", "c25s25", "i25r25", "c25r25", "i75s25", "c25s75", "i25r75", "c25r75"]

epochsTFRshape = 0,176,46,50

allBaselines = np.empty(shape=(epochsTFRshape))
# allSpec = np.empty(shape=(epochsTFRshape))
biggestTimeDim = 410  # 410 for D0057, need a more efficient way of finding this though.

for trialType in trialTypes:
    for epoch, t, name in zip(
            ("Stimulus"+trialType, "Stimulus"+trialType, "Response"+trialType),
            ((-0.5, 0), (-0.5, 2), (-0.5, 2)),
            ("baselineStimulus"+trialType, "Stimulus"+trialType, "Response"+trialType)):
        times = [None, None]
        times[0] = t[0] - 0.5
        times[1] = t[1] + 0.5
        trials = trial_ieeg(good, epoch, times, preload=True)
        outliers_to_nan(trials, outliers=10)
        spec = wavelet_scaleogram(trials, n_jobs=-2, decim=int(good.info['sfreq'] / 100))
        
        timeDim = spec.data.shape[3]
        # if timeDim > biggestTimeDim:
        #     biggestTimeDim = timeDim

        crop_pad(spec, "0.5s")
        print(name + ": " + str(spec.data.shape))
        if name == "baselineStimulus"+trialType:
            base = spec.copy()
            continue
        allBaselines = np.concatenate([allBaselines, base.data], axis=0)


   

Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
23 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 23 events and 3073 original time points ...


0 bad epochs dropped


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:    9.8s


Not setting metadata


[Parallel(n_jobs=-2)]: Done 176 out of 176 | elapsed:   11.4s finished


...

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:   22.6s


Not setting metadata


[Parallel(n_jobs=-2)]: Done 176 out of 176 | elapsed:   26.5s finished


Stimulusi75s25i75s25: (23, 176, 46, 255)
Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
23 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 23 events and 7169 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:   18.6s


In [30]:
# Rescale all the specs after baselines are concatenated
for trialType in trialTypes:
    for epoch, t, name in zip(
            ("Stimulus"+trialType, "Stimulus"+trialType, "Response"+trialType),
            ((-0.5, 0), (-0.5, 2), (-1, 2)),
            ("baselineStimulus"+trialType, "Stimulus"+trialType, "Response"+trialType)):
        times = [None, None]
        times[0] = t[0] - 0.5
        times[1] = t[1] + 0.5
        trials = trial_ieeg(good, epoch, times, preload=True)
        outliers_to_nan(trials, outliers=10)
        spec = wavelet_scaleogram(trials, n_jobs=-2, decim=int(good.info['sfreq'] / 100))
        print(trialType + ": " + str(spec.data.shape))
        timeDim = spec.data.shape[3]
        if timeDim > biggestTimeDim:
            biggestTimeDim = timeDim
        crop_pad(spec, "0.5s")
        if name == "baselineStimulus"+trialType:
            continue
        base = spec.copy()
        base.data = allBaselines
        spec_a = rescale(spec, base, copy=True, mode='ratio').average(
            lambda x: np.nanmean(x, axis=0), copy=True)
        spec_a._data = np.log10(spec_a._data) * 20
        fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
        spec_a.info['subject_info']['files'] = tuple(fnames)
        spec_a.info['bads'] = good.info['bads']
        filename = os.path.join(save_dir, f'{name}-tfrAvgAllBaseline.h5')
        mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
        spec_a.save(os.path.join(save_dir, f'{name}-avgAvgAllBaseline.fif'), overwrite=True)


Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
23 matching events found


No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 23 events and 3073 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:    8.5s


Not setting metadata
i75s25: (23, 176, 46, 154)
Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
23 matching events found
No baseline correction applied


[Parallel(n_jobs=-2)]: Done 176 out of 176 | elapsed:   10.0s finished


...

c25r75: (23, 176, 46, 359)
Applying baseline correction (mode: ratio)
Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
22 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 22 events and 8193 original time points ...
0 bad epochs

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:   22.1s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:   24.7s


Not setting metadata


[Parallel(n_jobs=-2)]: Done 176 out of 176 | elapsed:   29.0s finished


c25r75: (22, 176, 46, 410)
Applying baseline correction (mode: ratio)


In [31]:
base

<EpochsTFR | time : [-0.992188, 1.986328], freq : [2.079716, 1064.814640], epochs : 890, channels : 176, ~2.68 GB>

In [11]:
allBaselines = np.empty(shape=(0,176,46,50))
allBaselines = np.concatenate([allBaselines, base.data], axis=0)

### for a single trial type

In [ ]:
# trialType = "i75s25"
trialTypes = ["i75s25", "c75s25", "i75r25", "c75r25", "i75s75", "c75s75", "i75r75", "c75r75",
              "i25s25", "c25s25", "i25r25", "c25r25", "i75s25", "c25s75", "i25r75", "c25r75"]

allBaselines = np.empty(shape=(0,176,46,50))


for trialType in trialTypes:
    for epoch, t, name in zip(
        ("Stimulus"+trialType, "Stimulus"+trialType,  "Response"+trialType),
        ((-0.5, 0), (-0.5, 2), (-1, 2)),
        ("baselineStimulus"+trialType, "Stimulus"+trialType, "Response"+trialType)):
        times = [None, None]
        times[0] = t[0] - 0.5
        times[1] = t[1] + 0.5
        trials = trial_ieeg(good, epoch, times, preload=True)
        outliers_to_nan(trials, outliers=10)
        spec = wavelet_scaleogram(trials, n_jobs=-2, decim=int(
            good.info['sfreq'] / 100))
        crop_pad(spec, "0.5s")
        if name == "baselineStimulus"+trialType:
            base = spec.copy()
            continue
            
        spec_a = rescale(spec, base, copy=True, mode='ratio').average(
            lambda x: np.nanmean(x, axis=0), copy=True)
        spec_a._data = np.log10(spec_a._data) * 20
        fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
        spec_a.info['subject_info']['files'] = tuple(fnames)
        spec_a.info['bads'] = good.info['bads']
        filename = os.path.join(save_dir, f'{name}-tfr.h5')
        mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
        spec_a.save(os.path.join(save_dir, f'{name}-avg.fif'), overwrite=True)


### to average across all trial types and just look at differences between stimulus and response